
# Pretrained Model
## Imports

In [15]:
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
import numpy

In [16]:
# The directory where we will store our smaller dataset/
base_dir = "data/lab_01_split"

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')


Homer = 'homer_simpson'
Bart = 'bart_simpson'
Burns = 'charles_montgomery_burns'
Krusty = 'krusty_the_clown'
Lisa = 'lisa_simpson'
Milhouse = 'milhouse_van_houten'
Marge = 'marge_simpson'
Moe = 'moe_szyslak'
Ned = 'ned_flanders'
Principal = 'principal_skinner'


# Directory with our training images
train_homer = os.path.join(train_dir, Homer)
train_bart = os.path.join(train_dir, Bart)
train_burns = os.path.join(train_dir, Burns)
train_krusty = os.path.join(train_dir, Krusty)
train_lisa = os.path.join(train_dir, Lisa)
train_milhouse = os.path.join(train_dir, Milhouse)
train_marge = os.path.join(train_dir, Marge)
train_more = os.path.join(train_dir, Moe)
train_ned = os.path.join(train_dir, Ned)
train_principal = os.path.join(train_dir, Principal)

# Directory with our valiation images
val_homer = os.path.join(validation_dir, Homer)
val_bart = os.path.join(validation_dir, Bart)
val_burns = os.path.join(validation_dir, Burns)
val_krusty = os.path.join(validation_dir, Krusty)
val_lisa = os.path.join(validation_dir, Lisa)
val_milhouse = os.path.join(validation_dir, Milhouse)
val_marge = os.path.join(validation_dir, Marge)
val_more = os.path.join(validation_dir, Moe)
val_ned = os.path.join(validation_dir, Ned)
val_principal = os.path.join(validation_dir, Principal)

# Directory with our test images
test_homer = os.path.join(test_dir, Homer)
test_bart = os.path.join(test_dir, Bart)
test_burns = os.path.join(test_dir, Burns)
test_krusty = os.path.join(test_dir, Krusty)
test_lisa = os.path.join(test_dir, Lisa)
test_milhouse = os.path.join(test_dir, Milhouse)
test_marge = os.path.join(test_dir, Marge)
test_more = os.path.join(test_dir, Moe)
test_ned = os.path.join(test_dir, Ned)
test_principal = os.path.join(test_dir, Principal)


In [17]:
print(test_bart)

data/lab_01_split/test/bart_simpson


In [18]:
print('total training Homer images:', len(os.listdir(train_homer)))
print('total validation Homer images:', len(os.listdir(val_homer)))
print('total test Homer images:', len(os.listdir(test_homer)))

total training Homer images: 1572
total validation Homer images: 337
total test Homer images: 337


## 2. Preprocessing


In [19]:
# Rescale all images by 1./255
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

In [20]:
train_generator = train_datagen.flow_from_directory(
        train_dir, # target directory
        class_mode='categorical') 

Found 9663 images belonging to 10 classes.


In [21]:
validation_generator = validation_datagen.flow_from_directory(
        validation_dir, # validation directory
        class_mode='categorical')

Found 2072 images belonging to 10 classes.


In [22]:
#Check which images cause problems 
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

cnt= 0

path = Path(train_homer).rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)
            cnt+=1
            
# print(cnt)

In [23]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (32, 256, 256, 3)
labels batch shape: (32, 10)


# 3. Data Augmentation

In [31]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
    ]
)

# 4. Building the Model



In [33]:
model = keras.applications.ResNet50(
    weights='imagenet',
    input_shape=(256, 256, 3),
    include_top=False,
)

# freeze the conv-net structure
base_model.trainable = False
print("Model done")

Model done


In [32]:
inputs = keras.Input(shape=(256, 256, 3), name="input")
# Augment your inputs here!
x = data_augmentation(inputs)

# scale pixels here
scale_layer = keras.layers.Rescaling(scale=1/127.5, offset=-1)
x = scale_layer(x)

x = base_model(x, training=False)               # Set it to inference mode
x = keras.layers.GlobalAveragePooling2D()(x)    # Pooling
x = keras.layers.Dropout(0.4)(x)                # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049  

# 5. Training the model

In [37]:
earlystop = EarlyStopping(
    monitor='val_acc',
    min_delta=0.001,
    patience=10,
    verbose=1,
    mode='auto')

csv_logger = CSVLogger('training.log',
                       separator=',',
                       append=False)

In [38]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.9),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

Fitting the model

In [39]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=50,
      callbacks=[earlystop, csv_logger]
)



Epoch 1/3


ValueError: in user code:

    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
    